In [60]:
import pdf2image
from PIL import Image
import pytesseract
import re
import pandas as pd

In [45]:
def extract_mine_production_and_reserves(page_text):
    lines = page_text.split('\n')
    mineral = "".join(lines[1:3])
    data = []
    found_reserves_section = False
    start_capturing = False
    scrap = False
    for line in lines:
        line = line.strip()
        if line[-9:-1] == "Reserves":
            scrap = True
        if scrap:
            if '2017 2018' in line:
                start_capturing = True
                continue
            if start_capturing:
                if 'World total' in line:
                    data.append(line)
                    break
                if line and re.search(r'\d', line):
                    data.append(line)
    return data, mineral
    
def parse_data(mineral_data):
    data, mineral = mineral_data
    structured_data = []
    
    first_line = data[0]
    a_1 = re.split(r'\s{2,}|\s+', first_line)
    if re.search(r'\d', a_1[-1]) or len(a_1)<=6:
        reserve_data = True
    else:
        reserve_data = False
        
    for line in data:
        a = re.split(r'\s{2,}|\s+', line)
        
        if reserve_data:
            country = a[0:-3]
            country = " ".join(country)
            prod_2017 = a[-3]
            prod_2018 = a[-2]
            reserves = a[-1]
            structured_data.append([mineral, country, prod_2017, prod_2018, reserves])
        else:
            for i in range(1, len(a)+1):
                if not re.search(r'\d', a[-1]) and len(a)>5:
                    a.pop()
                else:
                    break
            country = a[0:-2]
            country = " ".join(country)
            prod_2017 = a[-2]
            prod_2018 = a[-1]
            structured_data.append([mineral, country, prod_2017, prod_2018, "NA"])
            
    return structured_data

In [310]:
total_data = {}
for page in images[24:]:
    page_text = pytesseract.image_to_string(page)
    data, mineral = extract_mine_production_and_reserves(page_text)
    if len(data)>1:    
        parsed_data = parse_data((data, mineral))
        total_data[mineral] = parsed_data
    else:
        continue

In [48]:
page_text = pytesseract.image_to_string(images[26])

In [54]:
extract_mine_production_and_reserves(page_text)

(['United States —_ _— 660,000',
  'Australia 3,120 3,100 7140,000',
  'Bolivia 2,700 2,700 310,000',
  'Burma 1,000 1,000 NA',
  'China 98,000 100,000 480,000',
  'Guatemala 25 25 NA',
  'Iran 300 300 NA',
  'Kazakhstan 700 700 NA',
  'Laos 340 300 NA',
  'Mexico 243 240 18,000',
  'Pakistan 60 60 NA',
  'Russia (recoverable) 14,400 14,000 350,000',
  'Tajikistan 14,000 14,000 50,000',
  'Turkey 2,000 2,000 100,000',
  'Vietnam 380 300 NA',
  'World total (rounded) 137,000 140,000 1,500,000'],
 'ANTIMONY')

In [58]:
parse_data(extract_mine_production_and_reserves(page_text))

[['ANTIMONY', 'United States', '—_', '_—', '660,000'],
 ['ANTIMONY', 'Australia', '3,120', '3,100', '7140,000'],
 ['ANTIMONY', 'Bolivia', '2,700', '2,700', '310,000'],
 ['ANTIMONY', 'Burma', '1,000', '1,000', 'NA'],
 ['ANTIMONY', 'China', '98,000', '100,000', '480,000'],
 ['ANTIMONY', 'Guatemala', '25', '25', 'NA'],
 ['ANTIMONY', 'Iran', '300', '300', 'NA'],
 ['ANTIMONY', 'Kazakhstan', '700', '700', 'NA'],
 ['ANTIMONY', 'Laos', '340', '300', 'NA'],
 ['ANTIMONY', 'Mexico', '243', '240', '18,000'],
 ['ANTIMONY', 'Pakistan', '60', '60', 'NA'],
 ['ANTIMONY', 'Russia (recoverable)', '14,400', '14,000', '350,000'],
 ['ANTIMONY', 'Tajikistan', '14,000', '14,000', '50,000'],
 ['ANTIMONY', 'Turkey', '2,000', '2,000', '100,000'],
 ['ANTIMONY', 'Vietnam', '380', '300', 'NA'],
 ['ANTIMONY', 'World total (rounded)', '137,000', '140,000', '1,500,000']]

In [319]:
parsed_data = parse_data(extract_mine_production_and_reserves(page_text))

Some Reserves data has footnote before the digits, the following check_list contains those abnormal data

In [345]:
check_list = []
for mineral in all_data:
    try:
        total_reserve = float(mineral[-1][4])
        for state in mineral:
            state_reserve = float(state[4])
            if state_reserve>total_reserve:
                check_list.append(state[0])
                break
    except:
        pass

In [362]:
columns = ["Mineral", "Country", "Production 2017", "Production 2018", "Reserves"]
df = pd.DataFrame(all_data[0], columns=columns)
for mineral in all_data[1:]:
    columns = ["Mineral", "Country", "Production 2017", "Production 2018", "Reserves"]
    df_cat = pd.DataFrame(mineral, columns=columns)
    df = pd.concat([df, df_cat], ignore_index=True)

In [364]:
check_list # modify the corresponding calues in csv

['ANTIMONY',
 'BAUXITE AND ALUMINA',
 'COBALT',
 'COPPER',
 'DIAMOND (INDUSTRIAL)',
 'GOLD',
 'LEAD',
 'LITHIUM',
 'POTASH',
 'SODA ASH',
 'TIN',
 'VANADIUM',
 'ZINC']

In [388]:
df

,Mineral,Country,Production 2017,Production 2018,Reserves
0,ANTIMONY,United States,—_,_—,660000
1,ANTIMONY,Australia,3120,3100,7140000
2,ANTIMONY,Bolivia,2700,2700,310000
3,ANTIMONY,Burma,1000,1000,NA
4,ANTIMONY,China,98000,100000,480000
...,...,...,...,...,...
776,ZINC,Mexico,674,650,20000
777,ZINC,Peru,1470,1600,21000
778,ZINC,Sweden,251,220,1400
779,ZINC,Other countries,2140,2300,33000


In [389]:
df.to_csv("mineral_data.csv", index=False)

In [64]:
a = pd.read_csv("mineral_data.csv")
#a.to_excel("data_xlsx", index=None, header=True)

In [71]:
a.to_excel("data.xlsx", index=None, header=True)